# 3.3 线性回归的简洁实现

In [1]:
import numpy as np 
import torch 

# 3.3.1 生成数据集

In [2]:
num_input = 2 
num_examples = 1000 

true_w = [2,-3.4]
true_b =4.2 

# [1000,2]
features = torch.tensor(np.random.normal(0,1, (num_examples, num_input)),
                       dtype = torch.float32)

features.shape 

torch.Size([1000, 2])

In [3]:
labels = true_w[0]*features[:,0]+ true_w[1]*features[:,1]+true_b
labels+= torch.tensor(np.random.normal(0, 0.01, size = labels.size()), dtype= torch.float32)
labels.shape 

torch.Size([1000])

In [4]:
labels

tensor([ 1.2626e+00,  4.4058e+00,  1.1486e+01,  5.0196e+00,  3.9994e+00,
         7.5720e+00,  5.6822e+00,  5.0596e+00,  6.6707e+00,  7.5861e+00,
         9.8118e+00,  3.5012e+00,  5.3696e+00,  9.7664e+00,  1.7827e+00,
         1.0070e+01,  1.3953e+00,  3.2899e+00,  1.1453e+01,  1.5385e+01,
         1.3400e+00,  1.8073e+00,  6.9806e+00,  7.6943e+00, -4.6515e+00,
         2.8089e+00,  1.9710e+00,  4.7650e+00,  8.8328e+00,  5.4200e-02,
        -3.8086e+00,  5.3170e+00,  4.8404e+00,  5.3249e+00,  7.6755e+00,
         6.3386e+00, -2.3789e+00,  2.9974e+00, -3.0395e+00,  9.1863e+00,
         5.6961e+00,  6.2492e+00,  4.1853e+00,  6.6389e+00,  3.6867e+00,
         3.1361e+00,  6.9297e+00,  1.6425e+00,  3.8440e+00,  1.9943e+00,
         4.1926e+00,  4.4000e+00,  8.2863e+00,  5.1585e+00, -2.6393e+00,
         4.8699e+00,  3.1367e+00,  1.1313e+00,  1.1087e+01, -1.1336e-01,
         3.1194e+00,  1.1694e+01, -1.5656e+00,  3.4128e+00,  2.8105e+00,
        -7.1335e-01,  7.7286e+00,  4.5420e+00,  4.8

## 3.3.2 读取数据

In [5]:
import torch.utils.data as Data

In [6]:
batch_size =10 
# features [1000,2], laabs: [1000]
# # 将训练数据的特征和标签组合
dataset = Data.TensorDataset(features, labels)
# # 随机读取小批量
data_iter = Data.DataLoader(dataset, batch_size, shuffle=True)

In [51]:
for x , y in data_iter:
    print(x,y)
    print("###########")
    print(x.shape, y.shape) # torch.Size([10, 2]) torch.Size([10])
    break

tensor([[-1.7043, -0.5162],
        [ 0.0744, -0.2027],
        [ 0.6107,  0.5866],
        [ 1.9397,  0.2799],
        [-0.0595, -1.2625],
        [-1.3042, -1.1817],
        [-0.8565, -0.3980],
        [-0.7218, -0.4118],
        [ 0.1065,  2.0282],
        [-0.1131, -0.1738]]) tensor([ 2.5392,  5.0493,  3.4128,  7.1245,  8.3606,  5.6189,  3.8440,  4.1498,
        -2.4848,  4.5590])
###########
torch.Size([10, 2]) torch.Size([10])


In [52]:
len(data_iter)

100

In [54]:
type(data_iter)

torch.utils.data.dataloader.DataLoader

## 3.3.3 定义模型

In [11]:
import torch.nn as nn

In [59]:
class LinearNet(nn.Module):
    def __init__(self, n_feature):
        super(LinearNet, self).__init__()
        self.linear = nn.Linear(n_feature, 1)
    # forward 定义前向传播
    def forward(self, x):
        y = self.linear(x)
        return y

In [60]:
net = LinearNet(num_input)
net

LinearNet(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)

In [61]:
for param in net.parameters():
    print(param)

Parameter containing:
tensor([[-0.0374,  0.3280]], requires_grad=True)
Parameter containing:
tensor([-0.3147], requires_grad=True)


In [62]:
for name, param in net.named_parameters():
    print(name)
    print(param)
    print("#########################")

linear.weight
Parameter containing:
tensor([[-0.0374,  0.3280]], requires_grad=True)
#########################
linear.bias
Parameter containing:
tensor([-0.3147], requires_grad=True)
#########################


In [63]:
net

LinearNet(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)

In [64]:
type(net)

__main__.LinearNet

In [ ]:
"""
注意 要进行下一步
net[0] 这样的操作 
直接 调用 net[0] 会报错
需要做的是 sequencial it 
"""

In [65]:
net = nn.Sequential(
nn.Linear(num_input,1))

In [66]:
net

Sequential(
  (0): Linear(in_features=2, out_features=1, bias=True)
)

In [67]:
net[0]

Linear(in_features=2, out_features=1, bias=True)

In [68]:
net[0].weight

Parameter containing:
tensor([[-0.0015,  0.6964]], requires_grad=True)

In [69]:
net[0].bias

Parameter containing:
tensor([-0.0931], requires_grad=True)

## 3.3.4 初始化模型参数

In [70]:
from torch.nn import init

init.normal_(net[0].weight, mean=0, std=0.01)
init.constant_(net[0].bias, val=0)  # 也可以直接修改bias的data: net[0].bias.data.fill_(0)

Parameter containing:
tensor([0.], requires_grad=True)

## 3.3.5 定义损失函数

In [44]:
loss = nn.MSELoss()

## 3.3.6 定义优化算法

In [45]:
import torch.optim as optim

optimizer = optim.SGD(net.parameters(), lr =0.001)
optimizer

SGD (
Parameter Group 0
    dampening: 0
    lr: 0.001
    momentum: 0
    nesterov: False
    weight_decay: 0
)

In [71]:
x.shape, y.shape 

(torch.Size([10, 2]), torch.Size([10]))

In [72]:
y

tensor([ 2.5392,  5.0493,  3.4128,  7.1245,  8.3606,  5.6189,  3.8440,  4.1498,
        -2.4848,  4.5590])

In [73]:
y.view(-1,1)

tensor([[ 2.5392],
        [ 5.0493],
        [ 3.4128],
        [ 7.1245],
        [ 8.3606],
        [ 5.6189],
        [ 3.8440],
        [ 4.1498],
        [-2.4848],
        [ 4.5590]])

## 3.3.7 训练模型

In [46]:
num_epochs = 300 

for epoch in range(1, num_epochs+1):
    for x, y in data_iter:
        output = net(x) # [10,2]->[10,1]
        logits = loss(output, y.view(-1,1))
        optimizer.zero_grad()
        logits.backward()
        optimizer.step()
        
    print("epoch: %d, loss: %f "% (epoch, logits.item()))

epoch: 1, loss: 19.923719 
epoch: 2, loss: 10.414871 
epoch: 3, loss: 7.089870 
epoch: 4, loss: 4.951368 
epoch: 5, loss: 2.818110 
epoch: 6, loss: 2.257079 
epoch: 7, loss: 1.049172 
epoch: 8, loss: 0.743617 
epoch: 9, loss: 1.030025 
epoch: 10, loss: 0.808908 
epoch: 11, loss: 0.494111 
epoch: 12, loss: 0.181955 
epoch: 13, loss: 0.197541 
epoch: 14, loss: 0.115160 
epoch: 15, loss: 0.088433 
epoch: 16, loss: 0.060374 
epoch: 17, loss: 0.028628 
epoch: 18, loss: 0.011631 
epoch: 19, loss: 0.016844 
epoch: 20, loss: 0.012737 
epoch: 21, loss: 0.004408 
epoch: 22, loss: 0.006156 
epoch: 23, loss: 0.001819 
epoch: 24, loss: 0.001377 
epoch: 25, loss: 0.000733 
epoch: 26, loss: 0.001241 
epoch: 27, loss: 0.000347 
epoch: 28, loss: 0.001066 
epoch: 29, loss: 0.000389 
epoch: 30, loss: 0.000338 
epoch: 31, loss: 0.000202 
epoch: 32, loss: 0.000153 
epoch: 33, loss: 0.000281 
epoch: 34, loss: 0.000201 
epoch: 35, loss: 0.000098 
epoch: 36, loss: 0.000172 
epoch: 37, loss: 0.000118 
epoch: 3

epoch: 297, loss: 0.000132 
epoch: 298, loss: 0.000077 
epoch: 299, loss: 0.000140 
epoch: 300, loss: 0.000114 


In [74]:
net

Sequential(
  (0): Linear(in_features=2, out_features=1, bias=True)
)

In [76]:
net[0]

Linear(in_features=2, out_features=1, bias=True)

In [75]:
dense = net[0]
dense

Linear(in_features=2, out_features=1, bias=True)

In [49]:
true_w, dense.weight 

([2, -3.4], Parameter containing:
 tensor([[ 2.0002, -3.3999]], requires_grad=True))

In [50]:
true_b, dense.bias

(4.2, Parameter containing:
 tensor([4.1996], requires_grad=True))